<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Загрузка-данных" data-toc-modified-id="Загрузка-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Загрузка данных</a></span><ul class="toc-item"><li><span><a href="#Импорт-библиотек" data-toc-modified-id="Импорт-библиотек-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Импорт библиотек</a></span></li><li><span><a href="#Чтение-файла-и-изучение-данных" data-toc-modified-id="Чтение-файла-и-изучение-данных-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Чтение файла и изучение данных</a></span></li></ul></li><li><span><a href="#Умножение-матриц" data-toc-modified-id="Умножение-матриц-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Умножение матриц</a></span></li><li><span><a href="#Алгоритм-преобразования" data-toc-modified-id="Алгоритм-преобразования-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Алгоритм преобразования</a></span></li><li><span><a href="#Проверка-алгоритма" data-toc-modified-id="Проверка-алгоритма-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Проверка алгоритма</a></span><ul class="toc-item"><li><span><a href="#Подготовка-данных" data-toc-modified-id="Подготовка-данных-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>Подготовка данных</a></span></li><li><span><a href="#Класс-линейной-регрессии" data-toc-modified-id="Класс-линейной-регрессии-4.2"><span class="toc-item-num">4.2&nbsp;&nbsp;</span>Класс линейной регрессии</a></span></li><li><span><a href="#Алгоритм-шифрования---дешифрования" data-toc-modified-id="Алгоритм-шифрования---дешифрования-4.3"><span class="toc-item-num">4.3&nbsp;&nbsp;</span>Алгоритм шифрования - дешифрования</a></span></li><li><span><a href="#Сравнение-с-линейной-регрессий-из-sklearn" data-toc-modified-id="Сравнение-с-линейной-регрессий-из-sklearn-4.4"><span class="toc-item-num">4.4&nbsp;&nbsp;</span>Сравнение с линейной регрессий из sklearn</a></span></li></ul></li><li><span><a href="#Вывод" data-toc-modified-id="Вывод-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Вывод</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

# Защита персональных данных клиентов

Вам нужно защитить данные клиентов страховой компании «Хоть потоп». Разработайте такой метод преобразования данных, чтобы по ним было сложно восстановить персональную информацию. Обоснуйте корректность его работы.

Нужно защитить данные, чтобы при преобразовании качество моделей машинного обучения не ухудшилось. Подбирать наилучшую модель не требуется.

## Загрузка данных

### Импорт библиотек

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

import warnings
warnings.filterwarnings('ignore')

### Чтение файла и изучение данных

In [2]:
insurance = pd.read_csv('/datasets/insurance.csv')
insurance.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
Пол                  5000 non-null int64
Возраст              5000 non-null float64
Зарплата             5000 non-null float64
Члены семьи          5000 non-null int64
Страховые выплаты    5000 non-null int64
dtypes: float64(2), int64(3)
memory usage: 195.4 KB


In [3]:
insurance.head(5)

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0
3,0,21.0,41700.0,2,0
4,1,28.0,26100.0,0,0


In [4]:
insurance.tail(5)

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
4995,0,28.0,35700.0,2,0
4996,0,34.0,52400.0,1,0
4997,0,20.0,33900.0,2,0
4998,1,22.0,32700.0,3,0
4999,1,28.0,40600.0,1,0


- Датафрейм содержит 4 признака и 1 целевой признак (страховые выплаты клиенту за последние 5 лет)
- Все признаки — количественные
- Пропуски в данных отсутствуют

Проверим данные на наличие дубликатов.

In [5]:
insurance.duplicated().sum()

153

Количество дубликатов - 3% от общего числа. Для данного случая не слишком критично, особенно если учесть, что признаков не так много и их вариативность достаточно низкая.

В столбцах "возраст" и "зарплата" изменим тип данных на целочисленный.

In [6]:
insurance = insurance.astype({'Возраст': np.int})
insurance = insurance.astype({'Зарплата': np.int})

In [7]:
insurance.head(5)

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41,49600,1,0
1,0,46,38000,1,1
2,0,29,21000,0,0
3,0,21,41700,2,0
4,1,28,26100,0,0


В данных нет аномалий, приведение типов улучшило датасет. 

## Умножение матриц

Обозначения:

- $X$ — матрица признаков (нулевой столбец состоит из единиц)

- $y$ — вектор целевого признака

- $P$ — матрица, на которую умножаются признаки

- $w$ — вектор весов линейной регрессии (нулевой элемент равен сдвигу)

Предсказания:

$$
a = Xw
$$

Задача обучения:

$$
w = \arg\min_w MSE(Xw, y)
$$

Формула обучения:

$$
w = (X^T X)^{-1} X^T y
$$

При помощи формул докажем, что значение предсказания $a$ не изменится. Основная сложность заключается в том, что $X$ - прямоугольная матрица. Для того, чтобы она стала обратимой (и квадратной), мы и вводим умножение на транспонированную. Важно отметить, что перемножать мы можем только совместимые матрицы. 

Заменим матрицу $X$ на матрицу $M$: $ M = XP $, где $P$ - обратимая матрица

Найдем новый вектор весов:
$$ 
w_1 = ((XP)^T (XP))^{-1} (XP)^T y 
$$ 
Воспользуемся свойством: $ (AB)^T = B^TA^T $ 
$$ 
w_1 = (P^TX^TXP)^{-1}P^TX^T y 
$$ 
$P$ - обратимая матрица, а значит, она квадратная, $X^TX$ - тоже квадратная матрица, используемое свойство: $ (AB)^{-1} = B^{-1} A^{-1} $ 
$$ 
w_1 = P^{-1}(X^TX)^{-1}(P^T)^{-1}P^TX^T y 
$$ 
$$ 
w_1 = P^{-1}w
$$
Таким образом, новое предсказание: 
$$ 
a_1 = Mw_1 
$$
$$ 
a_1 = XPP^{-1}w
$$
$$
a_1 = XEw
$$
$$
a_1 = Xw = a
$$
Итак, предсказание $a$ **не меняется** при умножении признаков на обратимую матрицу.



**Признаки умножают на обратимую матрицу. Изменится ли качество линейной регрессии?**

**Ответ:** Умножение признаков на обратимую матрицу не меняет качество линейной регрессии.

**Обоснование:** Веса изменились под новые значения. Признаки исходной и преобразованной  матрицы выражаются через коэффициенты-веса, поэтому качество линейной регресии не изменилось.

## Алгоритм преобразования

**Алгоритм**

- Создание случайной обратимой матрицы $ P $, размером по количеству столбцов features.

- Проверка матрицы на обратимость.

- Нам надо закодировать данные, матрица $ P $ будет ключем. Перемножим матрицу признаков на ключ $ M = XP $, как результат - матрица с закодированными данными.  

- Применение алгоритма для преобразованных признаков.

- Определение метрики R2 на изначальных и преобразованных данных.

**Обоснование**

Такой алгоритм поможет защитить данные: матрица-ключ формируется случайно, при этом кодирование не влияет на работу модели. Качество линейной регрессии не меняется, т.к. оба множителя в произведении выражают исходные, но через определенные коэффициенты.

Данные полностью восстановлены после кодирования. Алгоритм работает.

$$ 
a = Xw 
$$
$$ 
a_1 = XPP^{-1}w
$$
$$
a_1 = XEw
$$
$$
a_1 = Xw = a
$$
Итак, предсказание $a$ **не меняется**, - качество линейной регрессии не меняется.

## Проверка алгоритма

### Подготовка данных

In [19]:
# делим на признаки и целевой признак: features и target
features = insurance.drop('Страховые выплаты', axis=1)
target = insurance['Страховые выплаты']

print('Признаки', features.shape)
print('Целевой признак', target.shape)

Признаки (5000, 4)
Целевой признак (5000,)


In [20]:
# теперь необходимо создать случайную матрицу
np.random.seed(12345)
random_matrix = np.random.randint(10,  size=(features.shape[1], features.shape[1]))
random_matrix

array([[2, 5, 1, 4],
       [9, 5, 2, 1],
       [6, 1, 9, 7],
       [6, 0, 2, 9]])

In [21]:
# проверим на обратимость
np.linalg.inv(random_matrix)

array([[-0.11015193,  0.11533149, -0.02589779,  0.05628453],
       [ 0.18542818,  0.01588398, -0.00656077, -0.07907459],
       [-0.00517956, -0.02279006,  0.13984807, -0.10393646],
       [ 0.07458564, -0.0718232 , -0.01381215,  0.09668508]])

In [22]:
# перемножим исходную на обратную матрицы, должна получиться единичная матрица
random_matrix @ np.linalg.inv(random_matrix)

array([[ 1.00000000e+00, -5.55111512e-17,  6.93889390e-18,
        -5.55111512e-17],
       [ 1.38777878e-16,  1.00000000e+00,  1.90819582e-17,
         2.77555756e-17],
       [-1.38777878e-17, -1.66533454e-16,  1.00000000e+00,
         4.16333634e-17],
       [-9.71445147e-17, -5.55111512e-17,  5.20417043e-18,
         1.00000000e+00]])

In [23]:
# теперь умножим датафрейм на рандомную матрицу
encoded_matrix = features @ random_matrix
encoded_matrix

,0,1,2,3
0,297977,49810,446485,347254
1,228420,38230,342094,266055
2,126261,21145,189058,147029
3,250401,41805,375346,291939
4,156854,26245,234957,182732
...,...,...,...,...
4995,214464,35840,321360,249946
4996,314712,52570,471670,366843
4997,203592,34000,305144,237338
4998,196418,32815,294351,228953


### Класс линейной регрессии

In [24]:
# воспользуемся классом с двумя методами из тренажера
class LRegression:
    def fit(self, train_features, train_target):
        X = np.concatenate((np.ones((train_features.shape[0], 1)), train_features), axis=1)
        y = train_target
        w = np.dot(np.dot(np.linalg.inv(np.dot(X.T, X)), X.T), y)

        self.w = w[1:]
        self.w0 = w[0]
        print(w)

    def predict(self, test_features):
        return test_features.dot(self.w) + self.w0

In [25]:
# создадим и обучим модель, найдем предсказания на обучающей выборке, найдем R2
model = LRegression()
model.fit(features, target)
predictions = model.predict(features)
r2_score(target, predictions)

[-9.38235447e-01  7.92580563e-03  3.57083050e-02 -1.70081903e-07
 -1.35676627e-02]


0.42494550308169177

In [26]:
# снова обучим модель, но уже на новых признаках
# сравним R2 и проверм, действительно ли все работает
model = LRegression()
model.fit(encoded_matrix, target)
predictions = model.predict(encoded_matrix)
r2_score(target, predictions)

[-9.38235404e-01  2.48160299e-03  3.10971798e-03  5.55304671e-04
 -3.28532114e-03]


0.424945503081687

### Алгоритм шифрования - дешифрования

In [27]:
# напишем алгоритм, который будет зашифровывать исходную матрицу
def encode(features):
    random_matrix = np.random.normal(0, 1, (features.shape[1], features.shape[1]))
    encoded_matrix = features @ random_matrix
    return encoded_matrix


encode(features)

,0,1,2,3
0,-37527.345989,19366.812651,-25023.673029,45844.795455
1,-28734.041473,14841.061990,-19174.460758,35131.581691
2,-15875.371402,8201.517528,-10597.757394,19417.299966
3,-31566.761986,16281.591384,-21034.719901,38533.883200
4,-19739.248360,10191.295165,-13169.199342,24128.279919
...,...,...,...,...
4995,-27012.539012,13941.353904,-18009.719938,32995.369057
4996,-39658.397860,20458.590719,-26435.380971,48426.931565
4997,-25658.431577,13237.265237,-17100.283482,31327.679212
4998,-24744.993615,12770.438966,-16493.449547,30220.026029


In [28]:
# напишем функцию для дешифрования
def decode(encoded_matrix):
    decoded_features = round(abs(encoded_matrix.dot(np.linalg.inv(random_matrix))))
    for i in decoded_features.iloc[:, [0, 3]]:
        decoded_features[i] = decoded_features[i].astype(int)
    decoded_features.columns = features.columns
    return decoded_features

decode(encoded_matrix)

,Пол,Возраст,Зарплата,Члены семьи
0,1,41.0,49600.0,1
1,0,46.0,38000.0,1
2,0,29.0,21000.0,0
3,0,21.0,41700.0,2
4,1,28.0,26100.0,0
...,...,...,...,...
4995,0,28.0,35700.0,2
4996,0,34.0,52400.0,1
4997,0,20.0,33900.0,2
4998,1,22.0,32700.0,3


In [29]:
features

,Пол,Возраст,Зарплата,Члены семьи
0,1,41,49600,1
1,0,46,38000,1
2,0,29,21000,0
3,0,21,41700,2
4,1,28,26100,0
...,...,...,...,...
4995,0,28,35700,2
4996,0,34,52400,1
4997,0,20,33900,2
4998,1,22,32700,3


### Сравнение с линейной регрессий из sklearn

Теперь воспользуемся линейной регрессий из sklearn. Для оценки используем метрику R2.

In [30]:
# обучим модель из sklearn на исходной матрице features
sklearn_model = LinearRegression().fit(features, target)
pred = sklearn_model.predict(features)
print('Метрика R2 до преобразования данных:', r2_score(target, pred))

Метрика R2 до преобразования данных: 0.42494550308169177


In [31]:
# обучим модель из sklearn на кодированной матрице encoded_matrix
sklearn_model.fit(encoded_matrix, target)
pred = sklearn_model.predict(encoded_matrix)
print('Метрика R2 после преобразования данных:', r2_score(target, pred))

Метрика R2 после преобразования данных: 0.42494550308169654


Качество метрики R2 для матрицы до преобразования и матрицы после проебразования почти равны. При этом, они совпадают со значением метрики R2 для написанного класса линейной регресии, что подтверждает правильность его написания.

## Вывод

- При умножении признаков на обратимую матрицу, качество линейной регрессии не меняется. Это объясняется изменением весов под новые значения. 

- Алгоритм, предложенный для шифрования данных, справляется со своей задачей.

- Создан алгоритм для дешифрования данных.

- Зашифрованные даные можно использовать длая предсказаний.

- Реализованный алгоритм удовлетворяет задаче защиты персональных данных.